# Импорты

In [276]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn import svm
from sklearn.preprocessing import scale

# Подготовка данных

In [207]:
df = pd.read_csv('HepatitisCdata.csv')
df.head(3)

,Unnamed: 0,Category,Age,Sex,ALB,ALP,ALT,AST,BIL,CHE,CHOL,CREA,GGT,PROT
0,1,0=Blood Donor,32,m,38.5,52.5,7.7,22.1,7.5,6.93,3.23,106.0,12.1,69.0
1,2,0=Blood Donor,32,m,38.5,70.3,18.0,24.7,3.9,11.17,4.80,74.0,15.6,76.5
2,3,0=Blood Donor,32,m,46.9,74.7,36.2,52.6,6.1,8.84,5.20,86.0,33.2,79.3


In [208]:
df = df.drop(df.columns[[0, 1, 2]], axis=1)

In [209]:
df.isnull().sum() # в некоторых строках отсутсвуют значения

Sex      0
ALB      1
ALP     18
ALT      1
AST      0
BIL      0
CHE      0
CHOL    10
CREA     0
GGT      0
PROT     1
dtype: int64

In [210]:
df = df.dropna() # выкидываем эти строки

In [211]:
y = df.iloc[:,0]
y = y.to_numpy()
y[y == 'm'] = 1
y[y == 'f'] = 0
y = y.astype('int')

In [212]:
x = df.iloc[:,1:].to_numpy()

In [213]:
x = scale(x) # приводим признаки к одному масштабу для исключения доминирования признаков с большими значениями над другими. Для этого все признаки помещаются в одно метрическое пространство
x[:3]

array([[-0.54270146, -0.60323005, -0.90549379, -0.35545693, -0.20228893,
        -0.58177708, -1.91609052,  0.48033581, -0.48102205, -0.54078756],
       [-0.54270146,  0.08405364, -0.41138002, -0.27628268, -0.40928316,
         1.35499299, -0.52424085, -0.15140186, -0.4165134 ,  0.86256616],
       [ 0.91641697,  0.25394399,  0.46171419,  0.57331796, -0.28278669,
         0.29068302, -0.16962947,  0.08549977, -0.09212705,  1.38648489]])

In [214]:
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42) # делим и перемешиваем датасет на выорки

In [215]:
# y_train

# Обучение

## Метод решающих деревьев

In [223]:
cv = KFold(n_splits=10, shuffle=True, random_state=0)

In [231]:
model = DecisionTreeClassifier(random_state=0)
score = cross_val_score(model, x, y, cv = cv, scoring = 'roc_auc')
acc = np.mean(score)
acc

0.6779215731421729

## Метод "k ближайших соседей"

In [218]:
cv = KFold(n_splits=10, shuffle=True, random_state=0)

In [221]:
scores = []
for k in range(1,51):
    model = KNeighborsClassifier(k)
    scores.append(cross_val_score(model, x, y, cv = cv, scoring = 'roc_auc'))

scores[:3]

[array([0.67874693, 0.57776498, 0.69310345, 0.70289855, 0.62745098,
        0.74417249, 0.64097744, 0.69055944, 0.8015873 , 0.60263158]),
 array([0.74815725, 0.61981567, 0.70804598, 0.73792271, 0.73389356,
        0.72377622, 0.66917293, 0.68181818, 0.78174603, 0.66315789]),
 array([0.72911548, 0.60253456, 0.74252874, 0.79166667, 0.75770308,
        0.747669  , 0.69047619, 0.73310023, 0.77936508, 0.65723684])]

In [222]:
acc = np.mean(scores, axis=1)
print(f"Оптимальное число соседей: {acc.argmax() + 1}\nТочность: {acc.max()}")

Оптимальное число соседей: 18
Точность: 0.7981758134143394


## SGDClassifier

In [274]:
cv = KFold(n_splits=10, shuffle=True, random_state=0)

In [277]:
model = SGDClassifier(random_state=0)
score = cross_val_score(model, x, y, cv = cv, scoring = 'roc_auc')
acc = np.mean(score)
acc

0.7336151105131318

## SVM

In [ ]:
# -----------------Просто работа модели с параметрами, найденными методом тыка----------------------

In [268]:
cv = KFold(n_splits=10, shuffle=True, random_state=0)

In [295]:
model = svm.SVC(kernel='rbf', C=10., random_state=0)
score = cross_val_score(model, x, y, cv = cv, scoring = 'roc_auc')
acc = np.mean(score)
print(f"Точность: {acc}")

Точность: 0.8542845114198385


In [287]:
# -----------------Грид-сёрч(Авто-подбор наилучшего параметра для модели)----------------------

In [281]:
from sklearn.model_selection import GridSearchCV

In [283]:
grid = {'C': np.power(10., np.arange(-2, 3))}
cv = KFold(n_splits=10, shuffle=True, random_state = 0)

model = svm.SVC(kernel='linear', random_state = 0)

gs = GridSearchCV(model, grid, scoring = 'roc_auc', cv=cv)
gs.fit(x, y)

GridSearchCV(cv=KFold(n_splits=10, random_state=0, shuffle=True),
             estimator=SVC(kernel='linear', random_state=0),
             param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])},
             scoring='roc_auc')

In [285]:
gs.cv_results_

{'mean_fit_time': array([0.00606349, 0.00468621, 0.00698459, 0.02331932, 0.15616577]),
 'std_fit_time': array([0.00269218, 0.00030653, 0.00042061, 0.00427207, 0.03298656]),
 'mean_score_time': array([0.00146627, 0.00106313, 0.00104167, 0.00099344, 0.00109131]),
 'std_score_time': array([4.99121036e-04, 1.51762235e-04, 1.27113037e-04, 4.63661065e-05,
        6.46677203e-05]),
 'param_C': masked_array(data=[0.01, 0.1, 1.0, 10.0, 100.0],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 0.01}, {'C': 0.1}, {'C': 1.0}, {'C': 10.0}, {'C': 100.0}],
 'split0_test_score': array([0.81941032, 0.84889435, 0.85380835, 0.85503686, 0.85380835]),
 'split1_test_score': array([0.58064516, 0.63479263, 0.65322581, 0.6624424 , 0.66359447]),
 'split2_test_score': array([0.87701149, 0.87011494, 0.87011494, 0.86781609, 0.86781609]),
 'split3_test_score': array([0.79227053, 0.82246377, 0.83333333, 0.8321256 , 0.83091787]),
 'split4_tes

In [292]:
gs.best_params_

{'C': 10.0}

In [286]:
model.C = 10.0
score = cross_val_score(model, x, y, cv = cv, scoring = 'roc_auc')
acc = np.mean(score)
print(f"Точность: {acc}")

Точность: 0.8285419649533005


In [288]:
# -----------------Финалочка(Подбираем несколько параметров для модели SVM)----------------------

In [293]:
kernels = ['linear', 'poly', 'rbf', 'sigmoid']

In [294]:
grid = {'C': np.power(10., np.arange(-2, 3))}
cv = KFold(n_splits=10, shuffle=True, random_state = 0)

for k in kernels:
    model = svm.SVC(kernel=k, random_state = 0)

    gs = GridSearchCV(model, grid, scoring = 'roc_auc', cv=cv)
    gs.fit(x, y)
    
    model.C = gs.best_params_['C']
    
    score = cross_val_score(model, x, y, cv = cv, scoring = 'roc_auc')
    acc = np.mean(score)
    print(f"Наилучшая точность для ядра {k}, которая достигается при параметре С={gs.best_params_['C']}: {acc}")
    

Наилучшая точность для ядра linear, которая достигается при параметре С=10.0: 0.8285419649533005
Наилучшая точность для ядра poly, которая достигается при параметре С=10.0: 0.7298634204766016
Наилучшая точность для ядра rbf, которая достигается при параметре С=10.0: 0.8542845114198385
Наилучшая точность для ядра sigmoid, которая достигается при параметре С=0.1: 0.7851502691990419
